In [1]:
import os 
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
from datetime import datetime
from src.workflow.pipeline import culture, ephys

[2023-11-10 11:10:00,873][INFO]: Connecting milagros@db.datajoint.com:3306
[2023-11-10 11:10:01,508][INFO]: Connected milagros@db.datajoint.com:3306


### Select organoid and experiment

In [ ]:
culture.Experiment()

organoid_id e.g. O17,experiment_start_time,experiment_end_time,user,lineage_id de-identified code (e.g. hmau001),induction_culture_date,induction_culture_plate,post_induction_culture_date,post_induction_culture_plate,isolated_rosette_culture_date Date for isolating the rosette,isolated_rosette_culture_plate,organoid_culture_date,organoid_culture_plate,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O09,2023-05-18 12:25:00,2023-05-18 18:15:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys
O09,2023-05-18 18:15:00,2023-05-19 09:30:00,None,None,None,None,None,None,None,None,None,None,No Drug,nan,ephys
O09,2023-05-19 09:30:00,2023-05-19 15:35:00,None,None,None,None,None,None,None,None,None,None,Bicuculline,50.0,ephys
O09,2023-05-19 15:45:00,2023-05-20 15:40:00,None,None,None,None,None,None,None,None,None,None,Tetrodotoxin,1.0,ephys
O10,2023-05-18 12:25:00,2023-05-18 18:15:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys
O10,2023-05-18 18:15:00,2023-05-19 09:30:00,None,None,None,None,None,None,None,None,None,None,No Drug,nan,ephys
O10,2023-05-19 09:30:00,2023-05-19 15:35:00,None,None,None,None,None,None,None,None,None,None,Bicuculline,50.0,ephys
O10,2023-05-19 15:45:00,2023-05-20 15:40:00,None,None,None,None,None,None,None,None,None,None,Tetrodotoxin,1.0,ephys
O11,2023-05-18 12:25:00,2023-05-18 18:15:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys
O11,2023-05-18 18:15:00,2023-05-19 09:30:00,None,None,None,None,None,None,None,None,None,None,No Drug,nan,ephys


In [ ]:
culture.Experiment & "organoid_id='O13'" & "experiment_start_time='2023-06-08 19:05:00'"

organoid_id e.g. O17,experiment_start_time,experiment_end_time,user,lineage_id de-identified code (e.g. hmau001),induction_culture_date,induction_culture_plate,post_induction_culture_date,post_induction_culture_plate,isolated_rosette_culture_date Date for isolating the rosette,isolated_rosette_culture_plate,organoid_culture_date,organoid_culture_plate,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O13,2023-06-08 19:05:00,2023-06-08 23:35:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys


In [ ]:
# Modify this cell to insert sessions
session_info = dict(
    organoid_id="O13",
    experiment_start_time="2023-06-08 19:05:00",
    insertion_number=0,
    start_time="2023-06-08 19:10:00",
    end_time="2023-06-09 19:15:00",
    session_type="lfp", # "lfp" or "spike_sorting"
    probe="Q983",  # probe serial number
    port_id="A",  # Port ID ("A", "B", etc.)
    used_electrodes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17, 18, 19, 20, 21]  # electrodes used for the session
)

In [ ]:
# Insert the session
ephys.EphysSession.insert1(session_info, ignore_extra_fields=True, skip_duplicates=True)

ephys.EphysSessionProbe.insert1(session_info, ignore_extra_fields=True, skip_duplicates=True)

display(ephys.EphysSession())
display(ephys.EphysSessionProbe())

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
O13,2023-06-08 19:05:00,0,2023-06-08 19:10:00,2023-06-09 19:15:00,lfp
O17,2023-07-13 18:15:00,0,2023-07-13 18:30:00,2023-07-13 18:35:00,lfp


organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),port_id,"used_electrodes list of electrode IDs used in this session (if null, all electrodes are used)"
O13,2023-06-08 19:05:00,0,2023-06-08 19:10:00,2023-06-09 19:15:00,Q983,A,=BLOB=
O17,2023-07-13 18:15:00,0,2023-07-13 18:30:00,2023-07-13 18:35:00,Q983,A,=BLOB=


In [ ]:
(ephys.EphysSession & session_info).delete()

[2023-11-07 21:09:11,422][INFO]: Deleting 1 rows from `utah_organoids_ephys`.`ephys_session_probe`
[2023-11-07 21:09:11,758][INFO]: Deleting 1 rows from `utah_organoids_ephys`.`ephys_session`
[2023-11-07 21:09:13,102][INFO]: Deletes committed.


1